# 22_2 Regression - Tier End

Focus on Tier trips starting and ending at stations

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib
import holidays
from shapely import Point, wkt
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import shapiro, anderson, norm, expon, gamma
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


## Load and prepare data

In [2]:
data_month_double = '07'

In [3]:
# read final data
data = pd.read_csv("../../../20_Final_Data.csv")

/tmp/ipykernel_1607994/974816303.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../../20_Final_Data_NEW_TEST.csv")


In [4]:
tier = pd.read_parquet("../Data/12_tier_part2.parquet")
next = pd.read_parquet("../Data/15_nextbike_part2.parquet")

In [5]:
tier['trip_duration_seconds'] = tier['trip_duration'].dt.total_seconds()
next['trip_duration_seconds'] = next['trip_duration'].dt.total_seconds()

In [6]:
# transform geometry
data['station_point'] = data['station_point'].apply(wkt.loads)

In [7]:
data['buffer_zone'] = shapely.wkt.loads(data['buffer_zone'])

In [8]:
data = gpd.GeoDataFrame(data, geometry='station_point')

In [9]:
data = data.loc[:, ~data.columns.str.contains('Unnamed')]

In [10]:
#data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'], inplace=True)

In [11]:
data['actual_arrival_time'] = pd.to_datetime(data['actual_arrival_time'])
data['actual_departure_time'] = pd.to_datetime(data['actual_departure_time'])
data['scheduled_arrival_time'] = pd.to_datetime(data['scheduled_arrival_time'])
data['scheduled_departure_time'] = pd.to_datetime(data['scheduled_departure_time'])

In [12]:
data = gpd.GeoDataFrame(data, geometry='buffer_zone')

In [13]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 56 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   route_id                             int64         
 1   agency_id                            int64         
 2   route_short_name                     object        
 3   route_type                           int64         
 4   route_type_name                      object        
 5   agency_name                          object        
 6   service_id                           int64         
 7   trip_id                              object        
 8   trip_headsign                        object        
 9   direction_id                         int64         
 10  shape_id                             int64         
 11  stop_id                              int64         
 12  actual_arrival_time                  datetime64[ns]
 13  actual_departure_ti

In [14]:
data['date'] = pd.to_datetime(data['date'])

In [15]:
data['arrival_delay'] = pd.to_timedelta(data['arrival_delay'])
data['departure_delay'] = pd.to_timedelta(data['departure_delay'])

In [16]:
#NEW FEATURES
data['delay_category'] = data['arrival_delay'].apply(lambda x: 1 if x > pd.Timedelta(0) else 0 if x == pd.Timedelta(0) else -1)
data['cancelled_trip'] = np.where((data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()), 1, 0)

In [17]:
data['arrival_delay_float'] = data['arrival_delay'].astype(str)
data['arrival_delay_float'] = data['arrival_delay_float'].str.split(' ', n=2).str[-1]
data['arrival_delay_float'] = data['arrival_delay_float'].astype(str)
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float'])
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float']).dt.total_seconds() / 60

In [18]:
data['departure_delay_float'] = data['departure_delay'].astype(str)
data['departure_delay_float'] = data['departure_delay_float'].str.split(' ', n=2).str[-1]
data['departure_delay_float'] = data['departure_delay_float'].astype(str)
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float'])
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float']).dt.total_seconds() / 60

In [19]:
data['delay'] = data['departure_delay'].apply(lambda x: 1 if x > pd.Timedelta(0) else 0)
data['no_delay'] = data['departure_delay'].apply(lambda x: 1 if x == pd.Timedelta(0) else 0)
data['nan_delay'] = data['departure_delay'].apply(lambda x: 1 if pd.isna(x) else 0)

In [20]:
data['delay_span_int'] = 0

for index in data.index:
    if data['delay_span'][index] == '0':
        data['delay_span_int'][index] = 0
    if data['delay_span'][index] == '0-5':
        data['delay_span_int'][index] = 1
    if data['delay_span'][index] == '5-10':
        data['delay_span_int'][index] = 2
    if data['delay_span'][index] == '10-15':
        data['delay_span_int'][index] = 3
    if data['delay_span'][index] == '15-20':
        data['delay_span_int'][index] = 4
    if data['delay_span'][index] == '20-30':
        data['delay_span_int'][index] = 5
    if data['delay_span'][index] == '30-45':
        data['delay_span_int'][index] = 6
    if data['delay_span'][index] == '45-60':
        data['delay_span_int'][index] = 7
 
    

/tmp/ipykernel_1607994/1846020972.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['delay_span_int'][index] = 0
/tmp/ipykernel_1607994/1846020972.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [21]:
# new colum that sets trips that not happend to -1
data['arrival_delay_new'] = data['arrival_delay'].fillna(-1)
data['arrival_delay_float_new'] = data['arrival_delay_float'].fillna(-1)
# new colum that sets trips that not happend to -1
data['departure_delay_new'] = data['departure_delay'].fillna(-1)
data['departure_delay_float_new'] = data['departure_delay_float'].fillna(-1)

In [22]:
data['current_precipitation_volume'].fillna(0, inplace=True)

/tmp/ipykernel_1607994/969628070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['current_precipitation_volume'].fillna(0, inplace=True)


In [23]:
delay = data[data['departure_delay'] > pd.Timedelta(0)]


In [24]:
no_delay = data[data['departure_delay'] == pd.Timedelta(0)]

In [25]:
nan_delay = data[data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()]
#nan_delay = data[data['arrival_delay'].isna()]
nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_category,cancelled_trip,delay,no_delay,nan_delay,delay_span_int,arrival_delay_new,arrival_delay_float_new,departure_delay_new,departure_delay_float_new
41,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,255-551-008-2071.2.22:033200-34-157_C0741D60-B...,Bonn Hbf,1,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
42,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
43,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
44,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
45,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564854,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
2564856,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
2564857,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
2564858,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0


### Definition of functions

In [26]:
def calc_stats(data, column_name):

    #generated by Copilot

    sum = data[column_name].sum()

#Central Tendency

    rows = len(data.index)
    # Calculate mean (average)
    mean = data[column_name].mean()
    # Calculate median (middle value)
    median = data[column_name].median()
    # Calculate mode (most frequent value)
    mode = data[column_name].mode()
    #calc maximum
    max = data[column_name].max()
    #calc minimum
    min = data[column_name].min()

    # Calculate range
    data_range = data[column_name].max() - data[column_name].min()
    # Calculate coefficient of variation of range
    cvr = data_range / mean
    # Calculate coefficient of variation of mean
    cvm = mean / mean
    # Calculate coefficient of variation of median
    cvmed = median / mean
    # Calculate coefficient of variation of mode
    cvmode = mode / mean
    # Calculate coefficient of variation of standard deviation
   

# Variability
# Measure of how spread out the values in a data set are
    
    # Calculate standard deviation (average amount of variability in a dataset; tells how far each score lies from the mean; the larger the standard deviation, the more variable the data)
    std_dev = data[column_name].std()
    # Calculate variance (average squared deviation from the mean; square of standard deviation; units of varriance are much larger than those of a typical value in the dataset)
    variance = data[column_name].var()
    # Calculate coefficient of variation
    cv = std_dev / mean # standard deviation divided by the mean; measures the relative variability of a dataset; allows comparison of variability of datasets with different units of measurement
    # Calculate 25th percentile
    percentile_25 = data[column_name].quantile(0.25)
    # Calculate 75th percentile
    percentile_75 = data[column_name].quantile(0.75)
    # Calculate interquartile range
    iqr = percentile_75 - percentile_25 # difference between the 75th and 25th percentiles; measures the spread of the middle 50% of values in a dataset
    # Calculate coefficient of quartile deviation
    qd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of coefficient of variation
    cvcv = std_dev / mean
    # Calculate coefficient of variation of coefficient of quartile deviation
    cvqd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of range

#Measure of Shape

    # Calculate skewness - measure of how much the probability distribution of a random variable deviates from the normal distribution
    skewness = data[column_name].skew()
    
    # Calculate kurtosis
    kurtosis = data[column_name].kurtosis()

    # Calculate coefficient of skewness
    cs = skewness / std_dev
    # Calculate coefficient of kurtosis
    ck = kurtosis / std_dev
    # Calculate coefficient of variation of skewness
    cvs = skewness / std_dev
    # Calculate coefficient of variation of kurtosis
    cvk = kurtosis / std_dev
    # Calculate coefficient of variation of coefficient of skewness
    cvcs = skewness / std_dev
    # Calculate coefficient of variation of coefficient of kurtosis
    cvck = kurtosis / std_dev
    
    
    
    #return "column: " + str(column_name), "mean: " + str(mean), "median: " + str(median), "mode: " + str(mode), "std_dev: " + str(std_dev), "variance: " + str(variance), "skewness: "+ str(skewness), "kurtosis: " + str(kurtosis), "percentile_25: "+ str(percentile_25), "percentile_75: "+ str(percentile_75), "iqr: " + str(iqr), "range: "+ str(data_range), "cv"+ str(cv), "qd: "+str(qd), "cs: "+ str(cs), "ck: " + str(ck), "csv: "+ str(cvs), "cvk: "+ str(cvk), "cvcs: "+ str(cvcs), "cvck: " +str(cvck), "cvcv: "+ str(cvcv), "cvqd: "+ str (cvqd), "cvr: "+ str(cvr), "cvm: " + str(cvm), "cvmed: " + str(cvmed), "cvmode: "+ str(cvmode)

    return pd.DataFrame({'column': [column_name],
                         'number of rows': [rows],                       
                         'sum': [sum],
                         'mean': [mean],
                         'median': [median],
                         'mode': [mode],
                         'max': [max],
                         'min': [min],
                         'std_dev': [std_dev],
                         'variance': [variance],
                         'skewness': [skewness],
                         'kurtosis': [kurtosis],
                         'percentile_25': [percentile_25],
                         'percentile_75': [percentile_75],
                         'iqr': [iqr],
                         'range': [data_range],
                         'cv': [cv],
                         'qd': [qd],
                         'cs': [cs],
                         'ck': [ck],
                         'cvs': [cvs],
                         'cvk': [cvk],
                         'cvcs': [cvcs],
                         'cvck': [cvck],
                         'cvcv': [cvcv],
                         'cvqd': [cvqd],
                         'cvr': [cvr],
                         'cvm': [cvm],
                         'cvmed': [cvmed],
                         'cvmode': [cvmode]}).T

In [27]:
# create a shorter dataframe with most interesting columns - for better overview
def show_short_df(dataframe):
    delay_short = dataframe[['route_id', 'trip_id', 'stop_id', 'stop_name', 'scheduled_arrival_time', 'actual_arrival_time', 'scheduled_departure_time', 'actual_departure_time', 'arrival_delay', 'departure_delay', 'tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'buffer_zone', 'arrival_delay_float', 'departure_delay_float']]
    return delay_short

In [28]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["end_location"], crs=4326))
# drop geometry columns in wrong format
tier = tier.drop(columns=["start_location", "end_location"])
# replace geometry columns with correct format
tier['start_location'] = point_start
tier['end_location'] = point_end

In [29]:
# get the data from the tier / nextbike dataset
def check_micromobility_datasets(type, data):
    
    if type == 'tier':

        return_data = tier.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['tier_trips_id'].str.split(' ').explode().dropna():   # iterate over all tier ids in the dataframe
            
            id = tier[tier['tier_trips_id'] == int(float(item))]   # get the rows with the specific id
            return_data = pd.concat([return_data, id])  # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)
            
    if type == 'next':

        return_data = next.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['nextbike_trips_id'].str.split(' ').explode().dropna():   # iterate over all nextbike ids in the dataframe
            
            id = next[next['nextbike_trips_id'] == int(float(item))]  # get the rows with the specific id
            return_data = pd.concat([return_data, id]) # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)

    return return_data

In [30]:
def get_station_to_station(data, mode):
    
    if mode == 'tier':

        data = data[data['tier_trips_start_at_station_id'].notna() & data['tier_trips_end_at_station_id'].notna()]
        data = data[data['tier_trips_start_at_station_id'] != data['tier_trips_end_at_station_id']]
        data = data[data['tier_trips_start_at_station_id'] != 0]
        data = data[data['tier_trips_end_at_station_id'] != 0]
    t = check_micromobility_datasets('tier', data)
    n = check_micromobility_datasets('next', data)

    t = t[t['end_stop_id'].notna() & t['start_stop_id'].notna()]
    n = n[n['end_stop_id'].notna() & n['start_stop_id'].notna()]
    
    return t, n

In [31]:
# map the start and end locations of the trips on a map and connect them with a line

def map_buffer_zones(dataset):
    
    micromobility_data = check_micromobility_datasets('tier', dataset)
    
    # Create a folium map object
    m = folium.Map(location=[50.73743, 7.09821], zoom_start=12)

    # Iterate over each row in the micromobility data
    for index, row in micromobility_data.iterrows():
        # Get the start and end locations
        start_location = row['start_location']
        end_location = row['end_location']
        
        # Get the start and end stop names
        start_stop_name = row['start_stop_name']
        end_stop_name = row['end_stop_name']
        # Add markers for the start and end locations
        folium.Marker(location=[start_location.y, start_location.x], popup=start_stop_name).add_to(m)
        folium.Marker(location=[end_location.y, end_location.x], popup=end_stop_name).add_to(m)
        # Create a line connecting the start and end locations
        folium.PolyLine(locations=[[start_location.y, start_location.x], [end_location.y, end_location.x]], color='blue').add_to(m)
        
    # Display the map
    return m

In [32]:
def get_stats(data, var):

    if var == 'all':
        var = data.columns

    df = pd.DataFrame()

    for i in var:
        df = pd.concat([df, calc_stats(data, i)], axis=1)

    df.columns = df.iloc[0]
    df = df[1:]
    
    return df

In [33]:
def get_micro_stats(data, operator):
    
    if operator == 'tier':
        dataframe = get_stats(check_micromobility_datasets(operator, data), ['trip_duration_seconds', 'trip_distance_kilometers'])
    elif operator == 'next':
        dataframe = get_stats(check_micromobility_datasets(operator, data), ['trip_duration_seconds', 'trip_distance_kilometers'])
    
    return dataframe

In [34]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')
                 #Bbox = dict(facecolor = 'red', alpha =.8))

route_id', 'agency_id',
       'route_short_name', 'route_type', 'route_type_name', 'agency_name',
       'service_id', 'trip_id', 'trip_headsign', 'direction_id', 'shape_id',
       'stop_id', 'actual_arrival_time', 'actual_departure_time',
       'vrs_timestamp', 'stop_sequence', 'stop_headsign', 'stop_name',
       'station_point', 'buffer_zone', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'tier_trips_count',
       'tier_trips_id', 'nextbike_trips_count', 'nextbike_trips_id',
       'current_time', 'current_temp', 'current_feels_like',
       'current_cloudiness', 'current_visibility', 'current_wind_speed',
       'current_description', 'current_precipitation_volume', 'holiday',
       'holiday_NRW', 'holiday_name', 'holiday_name_NRW'

In [35]:
data.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_type',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id', 'stop_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'tier_trips_count',
       'tier_trips_id', 'tier_trips_end_at_station_count',
       'tier_trips_end_at_station_id', 'station_point', 'buffer_zone',
       'nextbike_trips_count', 'nextbike_trips_id',
       'nextbike_trips_end_at_station_count',
       'nextbike_trips_end_at_station_id', 'current_time', 'current_temp',
       'current_feels_like', 'current_cloudiness', 'current_visibility',
       'current_wind_speed', 'current_description',
       'current_precipitation_volume', 'arrival_delay_float',
       'departure_

## Def Features

In [36]:
y_string = 'tier_trips_end_at_station_count'

In [37]:
# create dummy variables for categories
delay_span_dummy = data[['delay_span']]
delay_span_dummy = pd.get_dummies(delay_span_dummy, columns=['delay_span'], drop_first=True)


weekday_dummy = data[['weekday']]
weekday_dummy = pd.get_dummies(weekday_dummy, columns=['weekday'], drop_first=True)


time_span_dummy = data[['time_span']]
time_span_dummy = pd.get_dummies(time_span_dummy, columns=['time_span'], drop_first=True)

data_slim = data.drop(columns=['route_id', 'agency_id', 'route_short_name',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'date', 'tier_trips_id', 'tier_trips_count',
       'tier_trips_end_at_station_id', 'station_point', 'buffer_zone',
       'nextbike_trips_count', 'nextbike_trips_id',
       'nextbike_trips_end_at_station_count', 'departure_delay', 'actual_arrival_time_float', 'time_span_int', 'delay_category', 'departure_delay_new',
       'nextbike_trips_end_at_station_id', 'current_time', 'current_feels_like', 'current_cloudiness', 'current_visibility',
       'current_wind_speed', 'current_description', 'arrival_delay_float',  'actual_departure_time_float',  'time_span_arrival', 'time_span_int_arrival', 'delay_span_int', 'arrival_delay_new', 'arrival_delay_float_new', 'arrival_delay_span'])

data_dummy = pd.get_dummies(data_slim, columns=['time_span', 'weekday', 'delay_span'], drop_first=True)

In [38]:
data_slim.columns

Index(['route_type', 'stop_id', 'service', 'weekday', 'transfer_stop',
       'tier_trips_end_at_station_count', 'current_temp',
       'current_precipitation_volume', 'departure_delay_float', 'delay_span',
       'weather_int', 'weekend', 'time_span', 'cancelled_trip', 'delay',
       'no_delay', 'nan_delay', 'departure_delay_float_new'],
      dtype='object')

In [39]:
data_dummy.columns

Index(['route_type', 'stop_id', 'service', 'transfer_stop',
       'tier_trips_end_at_station_count', 'current_temp',
       'current_precipitation_volume', 'departure_delay_float', 'weather_int',
       'weekend', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay',
       'departure_delay_float_new', 'time_span_afternoon',
       'time_span_early morning', 'time_span_evening',
       'time_span_evening rush', 'time_span_lunch', 'time_span_morning rush',
       'time_span_until midday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'delay_span_0-5', 'delay_span_10-15',
       'delay_span_15-20', 'delay_span_20-30', 'delay_span_30-45',
       'delay_span_45-60', 'delay_span_5-10'],
      dtype='object')

# Test for Model

In [51]:
# Varianz und Mittelwert der abhängigen Variable
# Var > Mean -> Poisson nicht gut (bei großer Abweichung)
mean = data[y_string].mean()
print(mean)
variance = data[y_string].var()
print(variance)
dispersion_ratio = variance / mean
print(dispersion_ratio)

0.015956030365778105
0.019311775089486893
1.2103120041000965


In [56]:
corr_matrix = data_dummy.corr()
corr_matrix

,route_type,stop_id,service,transfer_stop,tier_trips_end_at_station_count,current_temp,current_precipitation_volume,departure_delay_float,weather_int,weekend,...,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,delay_span_0-5,delay_span_10-15,delay_span_15-20,delay_span_20-30,delay_span_30-45,delay_span_45-60,delay_span_5-10
route_type,1.000000,0.060355,0.016856,-0.065857,-0.011605,0.027113,-0.001331,0.026825,-0.003618,-0.008933,...,0.004911,0.002984,0.004350,0.059182,0.005625,0.008414,0.008273,0.007988,0.005451,0.008172
stop_id,0.060355,1.000000,-0.001685,-0.015772,-0.036371,0.006173,-0.000459,-0.000539,-0.001419,-0.001467,...,0.001104,0.000740,0.001130,0.019224,0.030173,-0.004215,0.014147,-0.006972,-0.009670,-0.006247
service,0.016856,-0.001685,1.000000,-0.007632,-0.000848,0.050747,0.001262,-0.020214,-0.000763,-0.095210,...,0.027363,0.027405,0.027388,0.012627,-0.001942,-0.002099,-0.007796,-0.034543,-0.026784,-0.000539
transfer_stop,-0.065857,-0.015772,-0.007632,1.000000,0.024994,-0.005364,-0.000140,-0.001756,0.000614,0.006757,...,-0.001854,-0.001280,-0.001870,-0.043125,-0.005749,0.002988,0.011454,0.014965,-0.004798,-0.002561
tier_trips_end_at_station_count,-0.011605,-0.036371,-0.000848,0.024994,1.000000,0.027085,-0.006149,-0.003946,-0.009053,0.002237,...,-0.000558,0.001548,-0.002024,-0.022765,-0.001484,-0.000314,-0.001120,0.000959,0.017086,-0.001665
current_temp,0.027113,0.006173,0.050747,-0.005364,0.027085,1.000000,-0.049263,-0.016466,-0.142011,0.135245,...,-0.126978,0.078613,-0.106187,0.025606,-0.002046,0.000572,-0.001695,-0.021065,-0.014491,0.008926
current_precipitation_volume,-0.001331,-0.000459,0.001262,-0.000140,-0.006149,-0.049263,1.000000,-0.003632,0.685637,0.087984,...,0.027576,-0.066925,-0.042474,-0.006357,-0.000857,-0.000513,-0.000959,-0.002646,-0.002144,-0.000870
departure_delay_float,0.026825,-0.000539,-0.020214,-0.001756,-0.003946,-0.016466,-0.003632,1.000000,-0.005550,0.006572,...,-0.002835,-0.001682,-0.002564,0.044318,0.091897,0.200583,0.245863,0.384454,0.191557,0.058298
weather_int,-0.003618,-0.001419,-0.000763,0.000614,-0.009053,-0.142011,0.685637,-0.005550,1.000000,0.067783,...,0.068347,-0.056565,-0.062573,-0.007977,-0.000623,-0.000165,-0.001100,-0.004030,-0.002951,-0.002118
weekend,-0.008933,-0.001467,-0.095210,0.006757,0.002237,0.135245,0.087984,0.006572,0.067783,1.000000,...,-0.288432,-0.288875,-0.288693,0.011333,0.006153,0.003714,-0.000079,0.011069,0.006178,-0.007732


In [57]:
# Calculate the correlation matrix
corr_matrix = data_dummy.corr()

# Select the target variable (dependent variable)
target_variable = y_string

# Sort the correlation matrix by the absolute values of the correlation with the target variable
sorted_corr = corr_matrix[target_variable].abs().sort_values(ascending=False)

# Select the top k features with the highest correlation with the target variable
k = 10
selected_features = sorted_corr[1:k+1].index.tolist()

# Print the selected features
print("Selected features:")
print(selected_features)

Selected features:
['stop_id', 'no_delay', 'delay', 'current_temp', 'transfer_stop', 'delay_span_0-5', 'delay_span_45-60', 'time_span_afternoon', 'time_span_morning rush', 'route_type']


In [58]:
selected_features = data_dummy[['stop_id', 'no_delay', 'delay', 'transfer_stop', 'current_temp', 'delay_span_0-5', 'time_span_afternoon', 'delay_span_45-60', 'time_span_morning rush', 'route_type']]

# Linear Regression

### Best Models

In [73]:
X = selected_features
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog)
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564850
Model Family:                             Gaussian   Df Model:                           10
Link Function:                            Identity   Scale:                        0.019195
Method:                                       IRLS   Log-Likelihood:             1.4302e+06
Date:                             Fri, 13 Sep 2024   Deviance:                       49231.
Time:                                     18:12:33   Pearson chi2:                 4.92e+04
No. Iterations:                                  3   Pseudo R-squ. (CS):           0.006087
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [72]:
X = data_dummy.drop(columns=["tier_trips_end_at_station_count", "departure_delay_float"])
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog)
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564829
Model Family:                             Gaussian   Df Model:                           31
Link Function:                            Identity   Scale:                        0.019157
Method:                                       IRLS   Log-Likelihood:             1.4327e+06
Date:                             Fri, 13 Sep 2024   Deviance:                       49135.
Time:                                     18:12:20   Pearson chi2:                 4.91e+04
No. Iterations:                                  3   Pseudo R-squ. (CS):           0.008047
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

### Other

In [59]:
short_data = data[['departure_delay', 'tier_trips_end_at_station_count']].copy()
short_data['departure_delay'] = pd.to_numeric(short_data['departure_delay'])

In [60]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay', data=short_data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                      -0.008
Model:                                         OLS   Adj. R-squared:                 -0.008
Method:                              Least Squares   F-statistic:                       nan
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):                nan
Time:                                     18:09:36   Log-Likelihood:             1.4123e+06
No. Observations:                          2564861   AIC:                        -2.825e+06
Df Residuals:                              2564860   BIC:                        -2.825e+06
Df Model:                                        0                                         
Covariance Type:                         nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.9

In [61]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.000
Model:                                         OLS   Adj. R-squared:                  0.000
Method:                              Least Squares   F-statistic:                     23.71
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):           1.12e-06
Time:                                     18:09:37   Log-Likelihood:             8.3125e+05
No. Observations:                          1522668   AIC:                        -1.662e+06
Df Residuals:                              1522666   BIC:                        -1.662e+06
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                            coef    std err          t      P>|t|      [0.025   

In [62]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_new', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.002
Model:                                         OLS   Adj. R-squared:                  0.002
Method:                              Least Squares   F-statistic:                     25.30
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     18:11:14   Log-Likelihood:             1.4247e+06
No. Observations:                          2564861   AIC:                        -2.849e+06
Df Residuals:                              2564681   BIC:                        -2.847e+06
Df Model:                                      179                                         
Covariance Type:                         nonrobust                                         
                                                          coef    std err       

In [63]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float_new', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.000
Model:                                         OLS   Adj. R-squared:                  0.000
Method:                              Least Squares   F-statistic:                     19.24
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):           1.15e-05
Time:                                     18:11:16   Log-Likelihood:             1.4224e+06
No. Observations:                          2564861   AIC:                        -2.845e+06
Df Residuals:                              2564859   BIC:                        -2.845e+06
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                                coef    std err          t      P>|t|      [0.02

In [64]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ delay_span', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.001
Model:                                         OLS   Adj. R-squared:                  0.001
Method:                              Least Squares   F-statistic:                     297.6
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     18:11:28   Log-Likelihood:             1.4235e+06
No. Observations:                          2564861   AIC:                        -2.847e+06
Df Residuals:                              2564853   BIC:                        -2.847e+06
Df Model:                                        7                                         
Covariance Type:                         nonrobust                                         
                          coef    std err          t      P>|t|      [0.025     

In [65]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ cancelled_trip', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.000
Model:                                         OLS   Adj. R-squared:                  0.000
Method:                              Least Squares   F-statistic:                     35.96
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):           2.01e-09
Time:                                     18:11:30   Log-Likelihood:             1.4224e+06
No. Observations:                          2564861   AIC:                        -2.845e+06
Df Residuals:                              2564859   BIC:                        -2.845e+06
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.97

In [66]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ delay', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.001
Model:                                         OLS   Adj. R-squared:                  0.001
Method:                              Least Squares   F-statistic:                     2145.
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     18:11:31   Log-Likelihood:             1.4235e+06
No. Observations:                          2564861   AIC:                        -2.847e+06
Df Residuals:                              2564859   BIC:                        -2.847e+06
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-

In [67]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ no_delay', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.001
Model:                                         OLS   Adj. R-squared:                  0.001
Method:                              Least Squares   F-statistic:                     3014.
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     18:11:33   Log-Likelihood:             1.4239e+06
No. Observations:                          2564861   AIC:                        -2.848e+06
Df Residuals:                              2564859   BIC:                        -2.848e+06
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-

In [68]:
model = smf.ols(formula='tier_trips_end_at_station_count ~ nan_delay', data=data).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.000
Model:                                         OLS   Adj. R-squared:                 -0.000
Method:                              Least Squares   F-statistic:                    0.9992
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):              0.318
Time:                                     18:11:34   Log-Likelihood:             1.4224e+06
No. Observations:                          2564861   AIC:                        -2.845e+06
Df Residuals:                              2564859   BIC:                        -2.845e+06
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-

In [69]:
model = smf.ols(formula='tier_trips_count ~ nan_delay', data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       tier_trips_count   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6165
Date:                Fri, 13 Sep 2024   Prob (F-statistic):              0.432
Time:                        18:11:36   Log-Likelihood:             1.1162e+06
No. Observations:             2564861   AIC:                        -2.232e+06
Df Residuals:                 2564859   BIC:                        -2.232e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0198      0.000    155.942      0.0

In [70]:
X = data[['departure_delay_float_new', 'current_temp', 'current_precipitation_volume', 'service', 'delay_span_int', 'time_span_int', 'weekend', 'delay_category', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.OLS(y, exog)
result = poission_model.fit()
print(result.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.003
Model:                                         OLS   Adj. R-squared:                  0.003
Method:                              Least Squares   F-statistic:                     678.9
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     18:11:40   Log-Likelihood:             1.4262e+06
No. Observations:                          2564861   AIC:                        -2.852e+06
Df Residuals:                              2564849   BIC:                        -2.852e+06
Df Model:                                       11                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-

# Poisson Regression

#### Best Models

In [40]:
agg_station_date_10min = data.groupby(['stop_id', 'stop_name', pd.Grouper(key='scheduled_arrival_time', freq='10Min')]).sum(numeric_only=True)

In [41]:
agg_station_date_10min.isna().any()

route_id                               False
agency_id                              False
route_type                             False
service_id                             False
direction_id                           False
shape_id                               False
vrs_timestamp                          False
stop_sequence                          False
service                                False
transfer_stop                          False
tier_trips_count                       False
tier_trips_end_at_station_count        False
nextbike_trips_count                   False
nextbike_trips_end_at_station_count    False
current_temp                           False
current_feels_like                     False
current_cloudiness                     False
current_visibility                     False
current_wind_speed                     False
current_precipitation_volume           False
arrival_delay_float                    False
departure_delay_float                  False
actual_arr

In [42]:
agg_station_date_10min

route_id  agency_id  \
stop_id stop_name            scheduled_arrival_time                        
43      Bonn Propsthof Nord  2023-07-01 00:50:00           16          1   
                             2023-07-01 01:20:00           16          1   
                             2023-07-01 01:50:00           32          2   
                             2023-07-01 02:00:00           16          1   
                             2023-07-01 02:50:00           16          1   
...                                                       ...        ...   
9780    Bonn Georg-Elser-Weg 2023-07-31 22:50:00          604          6   
                             2023-07-31 23:10:00          604          6   
                             2023-07-31 23:20:00          604          6   
                             2023-07-31 23:40:00          604          6   
                             2023-08-01 00:10:00          604          6   

                                                     route_type  service_id  \
stop_id stop_name            scheduled_arrival_time                           
43      Bonn Propsthof Nord  2023-07-01 00:50:00              0       11525   
                             2023-07-01 01:20:00              0       11525   
                             2023-07-01 01:50:00              0       23061   
                             2023-07-01 02:00:00              0       11525   
                             2023-07-01 02:50:00              0       11525   
...                                                         ...         ...   
9780    Bonn Georg-Elser-Weg 2023-07-31 22:50:00              3          43   
                             2023-07-31 23:10:00              3          43   
                             2023-07-31 23:20:00              3          43   
                             2023-07-31 23:40:00              3          43   
                             2023-08-01 00:10:00              3          43   

                                                     direction_id  shape_id  \
stop_id stop_name            scheduled_arrival_time                           
43      Bonn Propsthof Nord  2023-07-01 00:50:00                0      3176   
                             2023-07-01 01:20:00                0      3203   
                             2023-07-01 01:50:00                0      6329   
                             2023-07-01 02:00:00                1      3230   
                             2023-07-01 02:50:00                0      3021   
...                                                           ...       ...   
9780    Bonn Georg-Elser-Weg 2023-07-31 22:50:00                1      5633   
                             2023-07-31 23:10:00                0      5377   
                             2023-07-31 23:20:00                1      5633   
                             2023-07-31 23:40:00                0      5377   
                             2023-08-01 00:10:00                0      5377   

                                                     vrs_timestamp  \
stop_id stop_name            scheduled_arrival_time                  
43      Bonn Propsthof Nord  2023-07-01 00:50:00               308   
                             2023-07-01 01:20:00              1304   
                             2023-07-01 01:50:00              7108   
                             2023-07-01 02:00:00              1304   
                             2023-07-01 02:50:00             14304   
...                                                            ...   
9780    Bonn Georg-Elser-Weg 2023-07-31 22:50:00            220312   
                             2023-07-31 23:10:00            213806   
                             2023-07-31 23:20:00            223304   
                             2023-07-31 23:40:00            214807   
                             2023-08-01 00:10:00            221806   

                                                     stop_sequence  service  \
stop_id stop_name 

In [43]:
poisson_model = smf.poisson('tier_trips_end_at_station_count ~ departure_delay_float_new', data=agg_station_date_10min).fit()
print(poisson_model.summary())

Optimization terminated successfully.
         Current function value: 0.143838
         Iterations 18
                                 Poisson Regression Results                                
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              1290080
Model:                                     Poisson   Df Residuals:                  1290078
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                  0.1650
Time:                                     18:09:10   Log-Likelihood:            -1.8556e+05
converged:                                    True   LL-Null:                   -2.2222e+05
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------

In [85]:
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564829
Model Family:                              Poisson   Df Model:                           31
Link Function:                                 Log   Scale:                          1.0000
Method:                                      lbfgs   Log-Likelihood:            -2.0400e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.3158e+05
Time:                                     18:53:27   Pearson chi2:                 3.31e+06
No. Iterations:                                 16   Pseudo R-squ. (CS):           0.007163
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [90]:
X = selected_features
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564850
Model Family:                              Poisson   Df Model:                           10
Link Function:                                 Log   Scale:                          1.0000
Method:                                      lbfgs   Log-Likelihood:            -2.0458e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.3275e+05
Time:                                     18:53:55   Pearson chi2:                 3.38e+06
No. Iterations:                                 16   Pseudo R-squ. (CS):           0.006710
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

#### Other

In [74]:
X = data[['departure_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                              Poisson   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1322e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.5002e+05
Time:                                     18:12:40   Pearson chi2:                 3.10e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          4.718e-07
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [75]:
X = data[['departure_delay_float_new']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                              Poisson   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1319e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4996e+05
Time:                                     18:12:47   Pearson chi2:                 5.67e+08
No. Iterations:                                  7   Pseudo R-squ. (CS):          2.192e-05
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [76]:
from sklearn.preprocessing import StandardScaler


X = delay_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564853
Model Family:                              Poisson   Df Model:                            7
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1221e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4801e+05
Time:                                     18:13:01   Pearson chi2:                 3.10e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          0.0007813
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [77]:
from sklearn.preprocessing import StandardScaler

X = data[['delay_category']]
y = data['tier_trips_count']

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:       tier_trips_count   No. Observations:              2564861
Model:                            GLM   Df Residuals:                  2564859
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -2.5316e+05
Date:                Fri, 13 Sep 2024   Deviance:                   4.1279e+05
Time:                        18:13:09   Pearson chi2:                 3.19e+06
No. Iterations:                     7   Pseudo R-squ. (CS):          0.0001907
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9308      0.004   -879.756      0.0

In [78]:
from sklearn.preprocessing import StandardScaler


X = time_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564853
Model Family:                              Poisson   Df Model:                            7
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1242e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4842e+05
Time:                                     18:13:23   Pearson chi2:                 3.08e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          0.0006232
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [79]:
from sklearn.preprocessing import StandardScaler


X = weekday_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564854
Model Family:                              Poisson   Df Model:                            6
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1290e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4939e+05
Time:                                     18:13:34   Pearson chi2:                 3.09e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          0.0002452
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [80]:
from sklearn.preprocessing import StandardScaler

X = data[['cancelled_trip']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                              Poisson   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1320e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4997e+05
Time:                                     18:13:41   Pearson chi2:                 3.10e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          1.705e-05
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [81]:

X = data[['nan_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                              Poisson   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1322e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.5002e+05
Time:                                     18:13:48   Pearson chi2:                 3.10e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          4.718e-07
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [82]:
X = data[['no_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                              Poisson   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1158e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4674e+05
Time:                                     18:13:54   Pearson chi2:                 3.07e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):           0.001277
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [83]:
X = data[['delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                              Poisson   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1184e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4726e+05
Time:                                     18:14:00   Pearson chi2:                 3.08e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):           0.001073
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [86]:
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
X

,route_type,stop_id,service,transfer_stop,current_temp,current_precipitation_volume,weather_int,weekend,cancelled_trip,delay,...,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,delay_span_0-5,delay_span_10-15,delay_span_15-20,delay_span_20-30,delay_span_30-45,delay_span_45-60,delay_span_5-10
0,3,687,1,False,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
1,3,1106,1,True,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
2,3,1102,1,False,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
3,3,1115,1,True,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
4,3,1500,1,True,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564856,3,1204,1,False,17.30,1.34,1,0,1,0,...,False,True,False,False,False,False,False,False,False,False
2564857,3,1140,1,False,17.30,1.34,1,0,1,0,...,False,True,False,False,False,False,False,False,False,False
2564858,3,1141,1,False,17.30,1.34,1,0,1,0,...,False,True,False,False,False,False,False,False,False,False
2564859,3,687,1,False,17.30,1.34,1,0,0,0,...,False,True,False,False,False,False,False,False,False,False


In [87]:
X = time_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564853
Model Family:                              Poisson   Df Model:                            7
Link Function:                                 Log   Scale:                          1.0000
Method:                                      lbfgs   Log-Likelihood:            -2.1242e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4842e+05
Time:                                     18:53:33   Pearson chi2:                 3.08e+06
No. Iterations:                                  4   Pseudo R-squ. (CS):          0.0006232
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [88]:
X = delay_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564853
Model Family:                              Poisson   Df Model:                            7
Link Function:                                 Log   Scale:                          1.0000
Method:                                      lbfgs   Log-Likelihood:            -2.1221e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4801e+05
Time:                                     18:53:39   Pearson chi2:                 3.09e+06
No. Iterations:                                  8   Pseudo R-squ. (CS):          0.0007813
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [89]:
X = weekday_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564854
Model Family:                              Poisson   Df Model:                            6
Link Function:                                 Log   Scale:                          1.0000
Method:                                      lbfgs   Log-Likelihood:            -2.1290e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.4939e+05
Time:                                     18:53:45   Pearson chi2:                 3.09e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          0.0002452
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

# Zero Inflated Poisson Regression

In [91]:
import statsmodels.api as sm

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['departure_delay_float_new']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.084672
         Iterations: 16
         Function evaluations: 19
         Gradient evaluations: 19
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.04939
Time:                                     18:54:46   Log-Likelihood:            -2.1717e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                                coef    std err          z      P>|z|      [0.025      0.975

In [92]:

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['delay_span_int']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.083725
         Iterations: 19
         Function evaluations: 21
         Gradient evaluations: 21
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.03764
Time:                                     18:56:06   Log-Likelihood:            -2.1474e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
---------

In [93]:

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['delay_category']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.084574
         Iterations: 16
         Function evaluations: 18
         Gradient evaluations: 18
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.04816
Time:                                     18:57:19   Log-Likelihood:            -2.1692e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
---------

In [94]:
data_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 35 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   route_type                       int64  
 1   stop_id                          int64  
 2   service                          int64  
 3   transfer_stop                    bool   
 4   tier_trips_end_at_station_count  int64  
 5   current_temp                     float64
 6   current_precipitation_volume     float64
 7   departure_delay_float            float64
 8   weather_int                      int64  
 9   weekend                          int64  
 10  cancelled_trip                   int64  
 11  delay                            int64  
 12  no_delay                         int64  
 13  nan_delay                        int64  
 14  departure_delay_float_new        float64
 15  time_span_afternoon              bool   
 16  time_span_early morning          bool   
 17  time_spa

In [95]:
X = data_dummy.drop(columns=[y_string, "departure_delay_float"])
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X.astype(float))

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.079029
         Iterations: 35
         Function evaluations: 45
         Gradient evaluations: 45


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564828
Method:                                        MLE   Df Model:                           32
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                 0.02055
Time:                                     19:01:10   Log-Likelihood:            -2.0270e+05
converged:                                   False   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
inflate_const                    1.4701        nan        nan        n

In [96]:
X = delay_span_dummy
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X)

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.083455
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 21
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564854
Method:                                        MLE   Df Model:                            6
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.03429
Time:                                     19:02:20   Log-Likelihood:            -2.1405e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
-------

In [97]:
X = time_span_dummy
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X)

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.083393
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564854
Method:                                        MLE   Df Model:                            6
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.03353
Time:                                     19:03:51   Log-Likelihood:            -2.1389e+05
converged:                                   False   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                     1.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
inflate_const               3.5235      0.005    664.729      0.000       3.513 

In [98]:
X = weekday_dummy
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X)

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.081915
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564855
Method:                                        MLE   Df Model:                            5
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01521
Time:                                     19:05:19   Log-Likelihood:            -2.1010e+05
converged:                                   False   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                     1.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
inflate_const         3.1409      0.005    585.226      0.000       3.130       3.151
weekda

In [99]:
X = selected_features
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X.astype(float))

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1331: RuntimeWarning: overflow encountered in exp
  return -np.exp(XB) +  endog*XB - gammaln(endog+1)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1508: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + offset + exposure)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1509: RuntimeWarning: overflow encountered in multiply
  return (self.endog - L)[:,None] * X
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1509: RuntimeWarning: invalid value encountered in multiply
  return (self.endog - L)[:,None] * X
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1074: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/

         Current function value: 0.094860
         Iterations: 0
         Function evaluations: 13
         Gradient evaluations: 2


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564851
Method:                                        MLE   Df Model:                            9
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                 -0.1756
Time:                                     19:06:41   Log-Likelihood:            -2.4330e+05
converged:                                   False   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                     1.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
inflate_const              0.1000      0.010     10.291      0.000       0.081    

In [100]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['delay_category']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.084574
         Iterations: 16
         Function evaluations: 18
         Gradient evaluations: 18
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.04816
Time:                                     19:08:10   Log-Likelihood:            -2.1692e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
---------

In [101]:

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['cancelled_trip']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.083766
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.03815
Time:                                     19:09:53   Log-Likelihood:            -2.1485e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
---------

In [102]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data['tier_trips_count'],data[['delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.098202
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:        tier_trips_count   No. Observations:              2564861
Model:             ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                            MLE   Df Model:                            0
Date:                 Fri, 13 Sep 2024   Pseudo R-squ.:                -0.02853
Time:                         19:11:37   Log-Likelihood:            -2.5188e+05
converged:                        True   LL-Null:                   -2.4489e+05
Covariance Type:             nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const     3.3313

In [103]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string],data[['no_delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.084505
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.04731
Time:                                     19:13:16   Log-Likelihood:            -2.1674e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
----------

In [104]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string],data[['nan_delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.083781
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.03833
Time:                                     19:14:54   Log-Likelihood:            -2.1489e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
----------

In [105]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string],data[['nan_delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.083781
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                           ZeroInflatedPoisson Regression Results                          
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                         ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                                        MLE   Df Model:                            0
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                -0.03833
Time:                                     19:16:24   Log-Likelihood:            -2.1489e+05
converged:                                    True   LL-Null:                   -2.0695e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
----------

# Negative Binomial Regression

### Best

In [118]:
X = selected_features
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564850
Model Family:                     NegativeBinomial   Df Model:                           10
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.0224e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   2.9783e+05
Time:                                     19:48:36   Pearson chi2:                 3.31e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):           0.006495
Covariance Type:                         nonrobust                                         
                             coef    std err          z      P>|z|      [0.025  

In [114]:
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564829
Model Family:                     NegativeBinomial   Df Model:                           31
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.0166e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   2.9667e+05
Time:                                     19:48:11   Pearson chi2:                 3.25e+06
No. Iterations:                                100   Pseudo R-squ. (CS):           0.006942
Covariance Type:                         nonrobust                                         
                                   coef    std err          z      P>|z|      [0

#### Other

In [106]:
X = data[['departure_delay_float_new']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1057e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1449e+05
Time:                                     19:16:29   Pearson chi2:                 4.49e+08
No. Iterations:                                  6   Pseudo R-squ. (CS):          2.119e-05
Covariance Type:                         nonrobust                                         
                                coef    std err          z      P>|z|      [0.02

In [107]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['cancelled_trip']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1057e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1450e+05
Time:                                     19:16:33   Pearson chi2:                 3.05e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          1.679e-05
Covariance Type:                         nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.97

In [108]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_span_int']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1010e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1355e+05
Time:                                     19:16:38   Pearson chi2:                 3.11e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          0.0003846
Covariance Type:                         nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.97

In [109]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_category']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1043e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1420e+05
Time:                                     19:16:42   Pearson chi2:                 3.05e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          0.0001313
Covariance Type:                         nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.97

In [110]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.0924e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1183e+05
Time:                                     19:16:46   Pearson chi2:                 3.03e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):           0.001057
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [111]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['no_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.0898e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1132e+05
Time:                                     19:16:50   Pearson chi2:                 3.02e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):           0.001254
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [112]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['nan_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564859
Model Family:                     NegativeBinomial   Df Model:                            1
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1059e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1454e+05
Time:                                     19:16:54   Pearson chi2:                 3.06e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          4.644e-07
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

In [115]:
X = delay_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564853
Model Family:                     NegativeBinomial   Df Model:                            7
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.0961e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1257e+05
Time:                                     19:48:17   Pearson chi2:                 3.05e+06
No. Iterations:                                  7   Pseudo R-squ. (CS):          0.0007662
Covariance Type:                         nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.

In [116]:
X = time_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564853
Model Family:                     NegativeBinomial   Df Model:                            7
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.0981e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1297e+05
Time:                                     19:48:22   Pearson chi2:                 3.03e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          0.0006133
Covariance Type:                         nonrobust                                         
                              coef    std err          z      P>|z|      [0.025 

In [117]:
X = weekday_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                        Generalized Linear Model Regression Results                        
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:                                         GLM   Df Residuals:                  2564854
Model Family:                     NegativeBinomial   Df Model:                            6
Link Function:                                 Log   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:            -2.1028e+05
Date:                             Fri, 13 Sep 2024   Deviance:                   3.1392e+05
Time:                                     19:48:27   Pearson chi2:                 3.04e+06
No. Iterations:                                  6   Pseudo R-squ. (CS):          0.0002414
Covariance Type:                         nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0

# Zero Inflated Negative Binomial

In [119]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['departure_delay_float_new']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3937: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3938: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


Optimization terminated successfully.
         Current function value: 0.242680
         Iterations: 10
         Function evaluations: 14
         Gradient evaluations: 14


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                  -2.012
Time:                                     19:54:14   Log-Likelihood:            -6.2244e+05
converged:                                    True   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                     1.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
inflate_const                -7.7162        nan        nan        nan       

In [120]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_span_int']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.080424
         Iterations: 34
         Function evaluations: 38
         Gradient evaluations: 38
                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                0.001941
Time:                                     20:00:21   Log-Likelihood:            -2.0628e+05
converged:                                    True   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                1.536e-176
                     coef    std err          z      P>|z|      [0.025      0.975]
---------

In [121]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_category']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.080524
         Iterations: 31
         Function evaluations: 34
         Gradient evaluations: 34
                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:               0.0006922
Time:                                     20:06:53   Log-Likelihood:            -2.0653e+05
converged:                                    True   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                 3.506e-64
                     coef    std err          z      P>|z|      [0.025      0.975]
---------

In [122]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['cancelled_trip']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.080573
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:               8.515e-05
Time:                                     20:15:32   Log-Likelihood:            -2.0666e+05
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                 2.982e-09
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
inflate_const     -2.9651      1.089     -2.723      0.006      -5.100      -0.831
const          

In [123]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.080125
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                0.005652
Time:                                     20:24:43   Log-Likelihood:            -2.0551e+05
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const    -3.0482      0.726     -4.199      0.000      -4.471      -1.625
const            -

In [124]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['no_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.080052
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                0.006558
Time:                                     20:33:14   Log-Likelihood:            -2.0532e+05
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const    -2.8258      0.290     -9.734      0.000      -3.395      -2.257
const            -

In [125]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['nan_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.080580
         Iterations: 34
         Function evaluations: 38
         Gradient evaluations: 38
                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                        MLE   Df Model:                            1
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:              -4.022e-07
Time:                                     20:42:39   Log-Likelihood:            -2.0668e+05
converged:                                    True   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                     1.000
                    coef    std err          z      P>|z|      [0.025      0.975]
----------

In [126]:
# Define the independent variables (X) and the dependent variable (y)
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:4329: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3937: RuntimeWarning: invalid value encountered in multiply
  a1 * np.log(a1) + y * np.log(mu) -
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3936: RuntimeWarning: invalid value encountered in subtract
  llf = (gammaln(y + a1) - gammaln(y + 1) - gammaln(a1) +
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3974: RuntimeWarning: divide by zero enc

         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564827
Method:                                        MLE   Df Model:                           33
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                     nan
Time:                                     21:16:06   Log-Likelihood:                    nan
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
inflate_const                   -0.8305        nan        nan        n

In [127]:
# Define the independent variables (X) and the dependent variable (y)
X = time_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.080328
         Iterations: 35
         Function evaluations: 40
         Gradient evaluations: 40


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564853
Method:                                        MLE   Df Model:                            7
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                0.003126
Time:                                     21:23:40   Log-Likelihood:            -2.0603e+05
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                8.178e-275
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
inflate_const              -2.2796      0.254     -8.974      0.000      -2.778 

In [128]:
# Define the independent variables (X) and the dependent variable (y)
X = delay_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.080260
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564853
Method:                                        MLE   Df Model:                            7
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                0.003972
Time:                                     21:31:20   Log-Likelihood:            -2.0586e+05
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
inflate_const       -3.1411      0.630     -4.988      0.000      -4.375      -1.907
const    

In [129]:
# Define the independent variables (X) and the dependent variable (y)
X = weekday_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.080492
         Iterations: 35
         Function evaluations: 40
         Gradient evaluations: 40


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564854
Method:                                        MLE   Df Model:                            6
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                0.001091
Time:                                     21:38:34   Log-Likelihood:            -2.0645e+05
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                 3.287e-94
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
inflate_const        -2.1531      0.242     -8.881      0.000      -2.628      -1.678
const 

In [130]:
# Define the independent variables (X) and the dependent variable (y)
X = selected_features
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:4329: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3937: RuntimeWarning: invalid value encountered in multiply
  a1 * np.log(a1) + y * np.log(mu) -
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3936: RuntimeWarning: invalid value encountered in subtract
  llf = (gammaln(y + a1) - gammaln(y + 1) - gammaln(a1) +
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3974: RuntimeWarning: divide by zero enc

         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                      ZeroInflatedNegativeBinomialP Regression Results                     
Dep. Variable:     tier_trips_end_at_station_count   No. Observations:              2564861
Model:               ZeroInflatedNegativeBinomialP   Df Residuals:                  2564850
Method:                                        MLE   Df Model:                           10
Date:                             Fri, 13 Sep 2024   Pseudo R-squ.:                     nan
Time:                                     21:50:18   Log-Likelihood:                    nan
converged:                                   False   LL-Null:                   -2.0668e+05
Covariance Type:                         nonrobust   LLR p-value:                       nan
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
inflate_const             -0.8001        nan        nan        nan         nan    

In [131]:
# Zero inflated in thesis erklären

# Multilinear Regression 

### Best

In [138]:
# ALL

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['tier_trips_end_at_station_count', 'departure_delay_float', 'current_temp', 'current_precipitation_volume', 'service', 'weekday', 'delay_span', 'time_span', 'weekend', 'delay_category', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float + current_temp + current_precipitation_volume + service + weekday + delay_span + time_span + weekend + delay_category + cancelled_trip + delay + no_delay + nan_delay', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.011
Model:                                         OLS   Adj. R-squared:                  0.011
Method:                              Least Squares   F-statistic:                     644.7
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     22:33:59   Log-Likelihood:             8.3958e+05
No. Observations:                          1522668   AIC:                        -1.679e+06
Df Residuals:                              1522641   BIC:                        -1.679e+06
Df Model:                                       26                                         
Covariance Type:                         nonrobust                                         
                                   coef    std err          t      P>|t|      [0

### Other

In [132]:
# DELAY FEATURES ONLY

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[[y_string, 'departure_delay_float_new', 'delay', 'no_delay', 'nan_delay', 'delay_span', 'cancelled_trip']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float_new + delay + no_delay + nan_delay + delay_span + cancelled_trip', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.005
Model:                                         OLS   Adj. R-squared:                  0.005
Method:                              Least Squares   F-statistic:                     1186.
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     21:50:27   Log-Likelihood:             1.4289e+06
No. Observations:                          2564861   AIC:                        -2.858e+06
Df Residuals:                              2564849   BIC:                        -2.858e+06
Df Model:                                       11                                         
Covariance Type:                         nonrobust                                         
                                coef    std err          t      P>|t|      [0.02

In [134]:
# DELAY FEATURES ONLY

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['tier_trips_end_at_station_count', 'departure_delay_float', 'delay', 'no_delay', 'nan_delay', 'delay_span', 'cancelled_trip']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float + delay + no_delay + nan_delay + delay_span + cancelled_trip', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.010
Model:                                         OLS   Adj. R-squared:                  0.010
Method:                              Least Squares   F-statistic:                     1624.
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     22:32:47   Log-Likelihood:             8.3851e+05
No. Observations:                          1522668   AIC:                        -1.677e+06
Df Residuals:                              1522658   BIC:                        -1.677e+06
Df Model:                                        9                                         
Covariance Type:                         nonrobust                                         
                            coef    std err          t      P>|t|      [0.025   

In [135]:
# TIME FEATURES

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['tier_trips_end_at_station_count', 'departure_delay_float_new', 'departure_delay_float', 'delay', 'no_delay', 'nan_delay', 'weekday', 'weekend', 'time_span']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float_new + delay + no_delay + nan_delay + time_span + weekday + weekend', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.002
Model:                                         OLS   Adj. R-squared:                  0.002
Method:                              Least Squares   F-statistic:                     350.8
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     22:33:02   Log-Likelihood:             1.4252e+06
No. Observations:                          2564861   AIC:                        -2.850e+06
Df Residuals:                              2564844   BIC:                        -2.850e+06
Df Model:                                       16                                         
Covariance Type:                         nonrobust                                         
                                 coef    std err          t      P>|t|      [0.0

In [136]:
# TIME FEATURES

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['tier_trips_end_at_station_count', 'departure_delay_float_new', 'departure_delay_float', 'delay', 'no_delay', 'nan_delay', 'weekday', 'weekend', 'time_span']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float_new + delay + no_delay + nan_delay + time_span + weekday + weekend', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.002
Model:                                         OLS   Adj. R-squared:                  0.002
Method:                              Least Squares   F-statistic:                     350.8
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     22:33:18   Log-Likelihood:             1.4252e+06
No. Observations:                          2564861   AIC:                        -2.850e+06
Df Residuals:                              2564844   BIC:                        -2.850e+06
Df Model:                                       16                                         
Covariance Type:                         nonrobust                                         
                                 coef    std err          t      P>|t|      [0.0

In [137]:
# ALL

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['tier_trips_end_at_station_count', 'departure_delay_float_new', 'current_temp', 'current_precipitation_volume', 'service', 'weekday', 'delay_span', 'time_span', 'weekend', 'delay_category', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='tier_trips_end_at_station_count ~ departure_delay_float_new + current_temp + current_precipitation_volume + service + weekday + delay_span + time_span + weekend + delay_category + cancelled_trip + delay + no_delay + nan_delay', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     tier_trips_end_at_station_count   R-squared:                       0.006
Model:                                         OLS   Adj. R-squared:                  0.006
Method:                              Least Squares   F-statistic:                     569.4
Date:                             Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                                     22:33:40   Log-Likelihood:             1.4304e+06
No. Observations:                          2564861   AIC:                        -2.861e+06
Df Residuals:                              2564832   BIC:                        -2.860e+06
Df Model:                                       28                                         
Covariance Type:                         nonrobust                                         
                                   coef    std err          t      P>|t|      [0